# Yoseph Yan, ypy4, February 8, 2024, Intelligent Autonomous Systems

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import io
from skimage.measure import label, regionprops
import os

In [2]:
# Define the paths to the folders
imagepath = 'trainingimgs'
orangepath = 'orangemasks'

orangepixels = []

# Loop through the files in the image folder
for imagefile in os.listdir(imagepath):
    # Full path to the file in the first folder
    imagefilepath = os.path.join(imagepath, imagefile)
    
    # Get name of the files in the mask folder
    orangemaskfile = imagefile.replace('.png', '')
    orangemaskfile = f"{orangemaskfile}_OrangeCone_mask.npy"
        
    # Full path to the corresponding file in the mask folder
    orangemaskpath = os.path.join(orangepath, orangemaskfile)
        
    # Check if the file is png and load the files
    if imagefile.endswith('.png'):
        image = io.imread(imagefilepath)
        
        orangemask = np.load(orangemaskpath)
        orange = image[orangemask]
        orangepixels.append(orange)
        
orangepixels = np.concatenate(orangepixels, axis=0)
covar = np.cov(orangepixels.T)

FileNotFoundError: [Errno 2] No such file or directory: 'trainingimgs'

In [6]:
# EM Algorithm for GMM
# Used the equations mentioned in lecture
n = orangepixels.shape[0]
def e_step(means, covars, cluster):
    xminusmu = orangepixels.reshape(-1, 1, 3) - means.reshape(1, -1, 3)
    g = np.exp(-0.5 * ((xminusmu.reshape(n, cluster, 1, 3) @ np.linalg.inv(covars)).reshape(n, cluster, 3) * xminusmu).sum(axis=2)) / ((2 * np.pi) ** 1.5 * np.sqrt(np.array([np.linalg.det(covars[i, :, :].reshape(3, 3)) for i in range(cluster)]))).reshape(1, cluster)
    z = g / g.sum(axis=1, keepdims=True)
    return z, g

def m_step(z, cluster):
    means = z.T @ orangepixels / z.sum(axis=0, keepdims=True).T
    xminusmu = orangepixels.reshape(-1, 1, 3) - means.reshape(1, -1, 3)
    covars = (xminusmu.reshape(n, cluster, 3, 1) * xminusmu.reshape(n, cluster, 1, 3) * z.reshape(n, cluster, 1, 1)).sum(axis=0) / z.sum(axis=0).reshape(cluster, 1, 1)
    return means, covars

def get_gmm_parameters(cluster, maxiterations = 1000, term = 1e-5):
    means = np.random.rand(cluster, 3) * 255
    prevmeans = means.copy()
    covars = np.array([covar + np.random.rand() * 1 for _ in range(cluster)])
    
    for _ in range(maxiterations):
        z, g = e_step(means, covars, cluster)
        means, covars = m_step(z, cluster)
        if np.linalg.norm(means - prevmeans) <= term:
            break
        prevmeans = means.copy()
    weight = 1 / np.max(g, axis=0)

    # Saves the parameters
    np.save('parameters/means.npy', np.array(means))
    np.save('parameters/covars.npy', np.array(covars))
    np.save('parameters/ws.npy', np.array(weight))

get_gmm_parameters(3)